In [1]:
import pandas as pd
import numpy as np
from math import ceil
import time
import glob,os
import matplotlib.pyplot as plt
import pickle
import pdb
from keras.preprocessing.sequence import pad_sequences

### Prepare data for LSTM: project: pedestrian trajecory prediction based on head orientation, previous trajectory and distance to cars

with open('/home/arash/ProjectVR/cleaneddata/ALLDATA', 'rb') as f:
    ALLDATA = pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/experiments', 'rb') as f:
    experiments = pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/DesignID', 'rb') as f:
    DesignID = pickle.load(f)

ALLDATAfortraj=ALLDATA.loc[-ALLDATA['starttime'].isin(['Err1','Err2','Err3','Err4','Err5'])]

# limit trajectory records to a time frame starting from 1 sec before cross to the end.
trajectory=[]
for n in range(len(experiments)):    
    if len(ALLDATAfortraj[ALLDATAfortraj['File']==experiments[n]['filename'][0]]['starttime'])>0:
        df=experiments[n][experiments[n]['timeelaspsed'].astype(float)>
                          (float(ALLDATAfortraj[ALLDATAfortraj['File']==
                                                experiments[n]['filename'][0]]['starttime'])-1)]
        df=df.drop(columns=['timehr','timemin','timesec',])
        trajectory.append(df)







# build time series data from pedestrian movements
userXtraj=[]
usero1traj=[]
usero2traj=[]
usero3traj=[]
userdisttraj=[]
userage=[]

for i in range(len(trajectory)):
    if len(trajectory[i][(trajectory[i]['status']=='u ') & (trajectory[i]['x'].astype(float)>650)])>0:
        oneroad=int(DesignID[DesignID['Card ID']==       #to see if the scenario is one road
                             int(ALLDATAfortraj[ALLDATAfortraj['File']
                                                ==trajectory[i].iloc[0,-1]]['Scenario'])]['Road Type_One way'])
        xarr=trajectory[i][(trajectory[i]['status']=='u ') & 
                           (trajectory[i]['x'].astype(float)>650)]['x'].values.astype(float)    
        # 650 threshold is the approximate x coordinate for the middle of the road, 
        #user trajectory in the end of experiment are removed
        
        
        #head orientation coordinates:
        headarr1=trajectory[i][(trajectory[i]['status']=='u ') & 
                           (trajectory[i]['x'].astype(float)>650)]['o1'].values.astype(float)  
        headarr2=trajectory[i][(trajectory[i]['status']=='u ') & 
                           (trajectory[i]['x'].astype(float)>650)]['o2'].values.astype(float)
        headarr3=trajectory[i][(trajectory[i]['status']=='u ') & 
                           (trajectory[i]['x'].astype(float)>650)]['o3'].values.astype(float)    
        
        
        
        age= ALLDATAfortraj[ALLDATAfortraj['File'] == trajectory[i].iloc[0,-1]][   #get age categories
                ['Age_9-12','Age_12-15', 'Age_15-18', 'Age_18 - 24',
                 'Age_25 - 29', 'Age_30 - 39', 'Age_40 - 49',
                 'Age_50 - 59', 'Age_60+']].values
        age=age.reshape(age.shape[1],)          #reshape to (9,)  9: number of age categories
        if not isinstance(age[0], str):
            age =age.astype(float) 

        
        
        firstpedindex=trajectory[i][(trajectory[i]['status']=='u ') & 
                           (trajectory[i]['x'].astype(float)>650)].index[0]  
                            #index for pedestrian 1st appearance
        pedindex=trajectory[i][(trajectory[i]['status']=='u ') & 
                           (trajectory[i]['x'].astype(float)>650)].index-firstpedindex   
                            #convert index to new format
        dist=[]
        for index, item in enumerate(pedindex):      #distance at every moment
            if index>0:
                bwped=trajectory[i].iloc[pedindex[index-1]:pedindex[index],:]     #data from between peds
                if len(bwped[bwped['status'].str.contains('L',na=False)].index)==0:   
                    #if no cars on left lane at the moment
                    distance=100
                else:            
                    if oneroad==0:
                        if len((bwped[(bwped['x'].astype(float)>650) & (bwped['status'] !='u ') &
                                      (bwped['y'].astype(float)> bwped.iloc[0,1])]['y']))==0:     
                            #check if any car that has not passed exists  
                            #1st cond: car in left lane, 2nd not a user data, 3rd: car has not passed ped yet
                            #differs for oneway and two-way roads
                            distance=100
                        else:
                            distance=min(bwped[(bwped['x'].astype(float)>650) & (bwped['status'] !='u ') &
                                               (bwped['y'].astype(float)> bwped.iloc[0,1])]['y']) - bwped.iloc[0,1] 
                    else:   #one-way roads
                        if len((bwped[(bwped['x'].astype(float)>650) & (bwped['status'] !='u ') &
                                      (bwped['y'].astype(float)< bwped.iloc[0,1])]['y']))==0:     
                            #check if any car that has not passed exists  
                            #1st cond: car in left lane, 2nd not a user data, 3rd: car has not passed ped yet
                            #differs for oneway and two-way roads
                            distance = 100
                        else:
                            distance = bwped.iloc[0,1] - max(bwped[(bwped['x'].astype(float)>650)
                                      & (bwped['status'] !='u ') & (bwped['y'].astype(float)< 
                                                                    bwped.iloc[0,1])]['y']) 
                dist.append(distance)


        userXtraj.append(xarr)
        usero1traj.append(headarr1)
        usero2traj.append(headarr2)
        usero3traj.append(headarr3)
        userdisttraj.append(dist)
        userage.append(age)

#Convert sequential tractory files to readable format for lstm. 

def sampleinput(userXtraj,usero1traj,usero2traj,usero3traj,userdisttraj, inputportion):
    X, o1, o2, o3, dist, y = list(), list(),list(), list(),list(), list()
    for i in range(len(userXtraj)):
        # find the end of this pattern
        xdistancecovered=max(userXtraj[i])-min(userXtraj[i])     #total distance covered by user from start to end of the expmeriment
        xinput=xdistancecovered*inputportion                    #distance that is used as input to the model                   
        breakpoint=min(userXtraj[i])+xinput
        n_steps=len([x for x in userXtraj[i] if x<breakpoint])
        
        # gather input and output parts of the pattern
        seq_x, seq_y = userXtraj[i][0:n_steps], userXtraj[i][n_steps:]
        seq_o1=usero1traj[i][0:n_steps]
        seq_o2=usero2traj[i][0:n_steps]
        seq_o3=usero3traj[i][0:n_steps]
        seq_dist=userdisttraj[i][0:n_steps]

        X.append(seq_x)
        o1.append(seq_o1)
        o2.append(seq_o2)
        o3.append(seq_o3)
        dist.append(seq_dist)
        y.append(seq_y)
        
    return X,o1,o2,o3 ,dist, y

inputportion = 0.5
# convert into input/output
X, o1, o2, o3, dist, y = sampleinput(userXtraj,usero1traj, usero2traj, usero3traj,userdisttraj, inputportion)

#check for ouliers
dur=[]
for i in range(len(X)):
    x=X[i].shape[0]+y[i].shape[0]
    dur.append(x)

dur=pd.DataFrame(dur,columns=['duration'])

dur.describe()

plt.hist(dur.iloc[:,0])

#indices of experiments when it takes more than ~15 seconds  to cross. 109 rows out of total 3424 removed
out=dur[dur['duration']>150].index

for i in sorted(out,reverse=True):
    del X[i]
    del o1[i]
    del o2[i]
    del o3[i]
    del dist[i]
    del y[i]
    del userage[i]

# converting list to array:
X=np.array(X)
o1=np.array(o1)
o2=np.array(o2)
o3=np.array(o3)
dist=np.array(dist)
y=np.array(y)
userage=np.array(userage)

#padding sequences to have same length by adding 0s:
Xpadded = pad_sequences(X, dtype='float32')       
o1padded = pad_sequences(o1,dtype='float32')
o2padded = pad_sequences(o2,dtype='float32')
o3padded = pad_sequences(o3,dtype='float32')
distpadded = pad_sequences(dist,dtype='float32')
ypadded = pad_sequences(y, padding='post',dtype='float32')

inputseq=[]
for i in range(Xpadded.shape[0]):
    mrg_input=np.transpose(np.vstack((Xpadded[i],o1padded[i],o2padded[i],o3padded[i],distpadded[i])))
    inputseq.append(mrg_input)
    

inputseq=np.array(inputseq)

allseqdata=[inputseq,ypadded]     #save all the sequence data to file


# save to pickle
with open('/home/arash/ProjectVR/cleaneddata/seqdata', 'wb') as f:
    pickle.dump(allseqdata, f)
   
with open('/home/arash/ProjectVR/cleaneddata/auxdata', 'wb') as f:
    pickle.dump(userage, f)

Using TensorFlow backend.
